In [1]:
import tensorflow as tf

2023-03-27 15:01:05.623773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 15:01:05.817273: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-27 15:01:06.433393: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fkirsch/miniconda3/envs/tensor/lib/
2023-03-27 15:01:06.433492: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_

In [2]:
%load_ext tensorboard
%tensorboard --logdir="logs/test/20230327-133812"

In [1]:
from env_wrapper2 import SelfPLayWrapper

# decide env
from tiktaktoe_env import TikTakToeEnv as GameEnv

import numpy as np
import datetime
import tensorflow as tf
import random as rnd


from agent import DQNAgent
from buffer import Buffer
from training import train_self_play_best

# seeds
seed = 42
#np.random.seed(seed)
#rnd.seed(seed) # otherwise always the same player starts
#tf.random.set_seed(seed)

#Subfolder from model
config_name = "test"
time_string = "20230327-133812"

model_path_best = f"model/{config_name}/{time_string}/best"

# Hyperparameter
iterations = 10001
INNER_ITS = 50 *2
BATCH_SIZE = 256 #512
#reward_function_adapting_agent = lambda d,r: tf.where(r==-0.1, tf.constant(0.1), tf.where(r==0.0,tf.constant(1.0),tf.where(r==1.0,tf.constant(-1.0), r)))
epsilon = 1
EPSILON_MIN = 0.01
EPSILON_DECAY = 0.995
POLYAK = 0.9
dropout_rate = 0
normalisation = True
BATCH_SIZE_SAMPLING = 512
SAMPLING = 2
AGENT_NUMBER = 2 # how many agents will play against each other while training
discount_factor_gamma = tf.constant(0.7)
unavailable_action_reward = False

# playing hyperparameter
index = 750

agents = []
train_writer = []
test_writer = []
env = SelfPLayWrapper(GameEnv)

agent = 1

best_train_path = f"logs/{config_name}/{time_string}/best_train{agent}"
train_writer.append(tf.summary.create_file_writer(best_train_path))

best_test_path = f"logs/{config_name}/{time_string}/best_test{agent}"
test_writer.append(tf.summary.create_file_writer(best_test_path))

model_path_best = f"model/{config_name}/{time_string}/best{agent}"

# create buffer
best_buffer = Buffer(capacity = 100000,min_size = 5000)

# create agent
agents.append(DQNAgent(env,
                       best_buffer, 
                       batch = BATCH_SIZE, 
                       model_path = model_path_best, 
                       polyak_update = POLYAK, 
                       inner_iterations = INNER_ITS, 
                       dropout_rate = dropout_rate, 
                       normalisation = normalisation, 
                       gamma = discount_factor_gamma))
    
agents[0].load_models(index)
env.set_opponent(agents[0])



2023-03-27 15:14:56.268592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 15:14:56.447567: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-27 15:14:57.043553: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/fkirsch/miniconda3/envs/tensor/lib/
2023-03-27 15:14:57.043691: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_

In [ ]:
env.reset()

player = 1 # rnd.randint(0,1) 
done=False

print("Start Player ", player)
if player: # if opponent starts
    env.opponent_starts()
env.render()

while(True):

    print("Your turn ")

    # choose action
    input_action = int(input())
    while(input_action not in env.available_actions):
        print("This action is not valid. Please try again. ")
        input_action = int(input())
    
    # do step, opponent is done automatically inside
    state, r, done = env.step(input_action)
    env.render()
    
    if(done):
        end = "won" if r==1 else "lost"
        print("You ", end) if r != 0 else print("Draw")
        break

    player = int(not player)

Start Player  1
Model results: 
 [[-1. -1. -1.]
 [-1. -1. -1.]
 [-1. -1. -1.]]
 Game                  Actions 
+---+---+---+     +---+---+---+
| ○ |   |   |     | 0 | 1 | 2 |
+---+---+---+     +---+---+---+
|   |   |   |     | 3 | 4 | 5 |
+---+---+---+     +---+---+---+
|   |   |   |     | 6 | 7 | 8 |
+---+---+---+     +---+---+---+

Your turn 


 2


Model results: 
 [[-1. -1. -1.]
 [-1. -1. -1.]
 [-1. -1. -1.]]
 Game                  Actions 
+---+---+---+     +---+---+---+
| ○ | ○ | ● |     | 0 | 1 | 2 |
+---+---+---+     +---+---+---+
|   |   |   |     | 3 | 4 | 5 |
+---+---+---+     +---+---+---+
|   |   |   |     | 6 | 7 | 8 |
+---+---+---+     +---+---+---+

Your turn 


 5


Model results: 
 [[-1. -1. -1.]
 [-1. -1. -1.]
 [-1. -1. -1.]]
 Game                  Actions 
+---+---+---+     +---+---+---+
| ○ | ○ | ● |     | 0 | 1 | 2 |
+---+---+---+     +---+---+---+
| ○ |   | ● |     | 3 | 4 | 5 |
+---+---+---+     +---+---+---+
|   |   |   |     | 6 | 7 | 8 |
+---+---+---+     +---+---+---+

Your turn 


 7


Model results: 
 [[-1. -1. -1.]
 [-1. -1. -1.]
 [-1. -1. -1.]]
 Game                  Actions 
+---+---+---+     +---+---+---+
| ○ | ○ | ● |     | 0 | 1 | 2 |
+---+---+---+     +---+---+---+
| ○ | ○ | ● |     | 3 | 4 | 5 |
+---+---+---+     +---+---+---+
|   | ● |   |     | 6 | 7 | 8 |
+---+---+---+     +---+---+---+

Your turn 
